<h1><center>Import Libraries & Data</center></h1>

In [2]:
import pandas as pd
import numpy as np
import regex as re
import seaborn as sns
import matplotlib as plt
import datetime as dt

In [3]:
# csv files
health = pd.read_csv("raw_data/health.csv")
covid = pd.read_csv("raw_data/covid.csv")
unemp = pd.read_csv("raw_data/unemp.csv")

# excel files
policy = pd.read_excel("raw_data/policy.xlsx")
gdp = pd.read_excel("raw_data/gdp.xlsx", header = 5)
income = pd.read_excel("raw_data/income.xlsx", header = 5)
pop = pd.read_excel("raw_data/pop.xlsx", header = 5)

/Users/hp/opt/anaconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


<h1><center>Data Cleaning</center><h1>

<h3><center>1. Mental Health Data</center></h3>

### Process
<font size="3">
<ol>
    <li>Add a new column date</li>
    <li>Remove irrelevant columns</li>
    <li>Pivot the data wider by making each of the three indicators "anxiety", "depression", "anxiety or depression" a variable</li>
</ol>

In [4]:
# the head of the initial data
health.head()

,Indicator,Group,State,Subgroup,Phase,Time Period,Time Period Label,Time Period Start Date,Time Period End Date,Value,Low CI,High CI,Confidence Interval,Quartile Range
0,Symptoms of Depressive Disorder,National Estimate,United States,United States,1,1,"Apr 23 - May 5, 2020",04/23/2020,05/05/2020,23.5,22.7,24.3,22.7 - 24.3,NaN
1,Symptoms of Depressive Disorder,By Age,United States,18 - 29 years,1,1,"Apr 23 - May 5, 2020",04/23/2020,05/05/2020,32.7,30.2,35.2,30.2 - 35.2,NaN
2,Symptoms of Depressive Disorder,By Age,United States,30 - 39 years,1,1,"Apr 23 - May 5, 2020",04/23/2020,05/05/2020,25.7,24.1,27.3,24.1 - 27.3,NaN
3,Symptoms of Depressive Disorder,By Age,United States,40 - 49 years,1,1,"Apr 23 - May 5, 2020",04/23/2020,05/05/2020,24.8,23.3,26.2,23.3 - 26.2,NaN
4,Symptoms of Depressive Disorder,By Age,United States,50 - 59 years,1,1,"Apr 23 - May 5, 2020",04/23/2020,05/05/2020,23.2,21.5,25.0,21.5 - 25.0,NaN


In [5]:
# change column names
health.columns = health.columns.str.replace(" ", "_")

# add new column date
start = pd.to_datetime(health["Time_Period_Start_Date"])
end = pd.to_datetime(health["Time_Period_End_Date"])
health["Date"] = start + (end - start) / 2
health["Date"] = [value.date() for value in health["Date"]]

# exclude irrelevant columns
health = health[["Indicator", "Group", "State", "Subgroup", "Date", "Value"]]

# pivot wider
health = health.pivot(index = ["Group", "State", "Subgroup", "Date"], columns = "Indicator", values = "Value")
health = health.reset_index()

# change column names
health.columns = health.columns.str.replace(" ", "_")
health.rename(columns = {
    "Symptoms_of_Anxiety_Disorder": "Pct_Anxiety",
    "Symptoms_of_Depressive_Disorder": "Pct_Depression",
    "Symptoms_of_Anxiety_Disorder_or_Depressive_Disorder": "Pct_Anxiety_Or_Depression"
}, inplace = True)

# reorder columns
health = pd.DataFrame({
    "State": health["State"],
    "Date": health["Date"],
    "Group": health["Group"],
    "Subgroup": health["Subgroup"],
    "Pct_Anxiety": health["Pct_Anxiety"],
    "Pct_Depression": health["Pct_Depression"],
    "Pct_Anxiety_Or_Depression": health["Pct_Anxiety_Or_Depression"]
})

In [6]:
# the final data
health.head()

,State,Date,Group,Subgroup,Pct_Anxiety,Pct_Depression,Pct_Anxiety_Or_Depression
0,United States,2020-04-29,By Age,18 - 29 years,40.2,32.7,46.8
1,United States,2020-05-09,By Age,18 - 29 years,42.0,36.2,47.4
2,United States,2020-05-16,By Age,18 - 29 years,38.6,36.7,47.7
3,United States,2020-05-23,By Age,18 - 29 years,39.1,36.7,46.6
4,United States,2020-05-30,By Age,18 - 29 years,42.2,36.8,49.3


<h3><center>2. Mental Health Data Grouped by State</center></h3>

### Process
<font size="3">
<ol>
    <li>Select only observations grouped by state</li>
    <li>For merging purposes, remove day information from variable "Date" so that every value has only year and month</li>
    <li>Take the average value of all months with multiple observations</li>
<ol>

In [7]:
# select only observations grouped by state
health_state = health.query('Group == "By State"')

# remove day information 
health_state["Date"] = pd.to_datetime(health["Date"]).dt.to_period("M")

# take the average of each month with multiple values
temp_df = health_state.groupby(["State", "Date"])["Pct_Anxiety", "Pct_Depression", "Pct_Anxiety_Or_Depression"].mean()
health_state = temp_df.reset_index()


<ipython-input-7-109b399eb527>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  health_state["Date"] = pd.to_datetime(health["Date"]).dt.to_period("M")
<ipython-input-7-109b399eb527>:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp_df = health_state.groupby(["State", "Date"])["Pct_Anxiety", "Pct_Depression", "Pct_Anxiety_Or_Depression"].mean()


In [8]:
# the final data
health_state.head(5)

,State,Date,Pct_Anxiety,Pct_Depression,Pct_Anxiety_Or_Depression
0,Alabama,2020-04,25.600,18.600000,30.300
1,Alabama,2020-05,26.175,22.850000,30.525
2,Alabama,2020-06,32.500,27.100000,37.225
3,Alabama,2020-07,36.400,30.566667,41.000
4,Alabama,2020-08,30.600,27.700000,37.200


<h3><center>3. Covid Policy Data</center></h3>

### Process
<font size="3">
<ol>
    <li>Remove irrelevant columns that do not have date information</li>
    <li>Pivot the dataset longer so that each row corresponds to one policy</li>
    <li>Add two new columns "Policy_Category" and "Policy_Stage"</li>
    <li>Modify columns names & data types & sort data</li>
<ol>

In [9]:
# the head of the initial data
policy.head()

,STATE,POSTCODE,FIPS,STEMERG,STEMERGEND,STEMERG2,CLSCHOOL,CLDAYCR,OPNCLDCR,CLNURSHM,...,MINWAGEMAR2019,MINWAGEJUL2019,MINWAGEOCT2019,MINWAGEJAN2020,MINWAGEJUL2020,MINWAGESEP2020,MINWAGEOCT2020,TIPMINWAGE2020,MINWAGE2021,SMALLBUSMINWAGE
0,State,State Abbreviation,FIPS Code,State of emergency issued,State of emergency lifted,State of emergency reinstated,Date closed K-12 public schools,Closed day cares,Reopen day cares,Date banned visitors to nursing homes,...,Mar 29 2019 Minimum Wage,Jul 1 2019 Minimum Wage,Oct 1 2019 Minimum Wage,Jan 1 2020 Minimum Wage,Jul 1 2020 Minimum Wage,Sep 1 2020 Minimum Wage,Oct 1 2020 Minimum Wage,2020 Minimum Wage for Tipped Workers,2021 Minimum Wage,Different Minimum Wage for Smaller Businesses
1,category,NaN,NaN,state_of_emergency,state_of_emergency,state_of_emergency,physical_distance_closure,physical_distance_closure,Reopening,physical_distance_closure,...,minimum_wage,minimum_wage,minimum_wage,minimum_wage,minimum_wage,minimum_wage,minimum_wage,minimum_wage,minimum_wage,minimum_wage
2,type,note,note,start,end,start,start,start,end,start,...,quantity,quantity,quantity,quantity,quantity,quantity,quantity,quantity,quantity,attribute
3,unit,text,attribute,date,date,date,date,date,date,date,...,dollars,dollars,dollars,dollars,dollars,dollars,dollars,dollars,dollars,flag
4,Alabama,AL,1,2020-03-13 00:00:00,2021-07-06 00:00:00,2021-08-13 00:00:00,2020-03-20 00:00:00,2020-03-20 00:00:00,2020-05-23 00:00:00,2020-03-19 00:00:00,...,7.25,7.25,7.25,7.25,7.25,7.25,7.25,2.13,7.25,0


In [10]:
# save postcode for later use
policy = pd.read_excel("raw_data/policy.xlsx")
postcode = pd.DataFrame({"State": policy["STATE"], "Postcode": policy["POSTCODE"]})
postcode = postcode.drop(axis = 0, index = [0, 1, 2, 3]).reset_index(drop = True)
postcode.head()

,State,Postcode
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [11]:
# exclude irrelevant columns
for column in policy.columns[3:]:
    if policy[column][3] != "date":
        policy.drop(axis = 1, labels = column, inplace = True)

# create a dictionary that maps each policy to the corresponding category and stage 
category_dict = {}
stage_dict = {}
for column in policy.columns[3:]:
    category_dict[column] = policy[column][1]
    stage_dict[column] = policy[column][2]

# remove all irrelevant rows and columns
policy = policy.drop(axis = 0, index = [0, 1, 2, 3])
policy = policy.drop(axis = 1, labels = ["POSTCODE", "FIPS"])

# pivot data longer
policy = pd.melt(policy, id_vars = "STATE", var_name = "Policy", value_name = "Date")
       
# add two new columns 
policy["Policy_Category"] = [category_dict[value] for value in policy["Policy"]]
policy["Policy_Stage"] = [stage_dict[value] for value in policy["Policy"]]

# remove all rows that are not formatted incorrected then convert the Date values to datetime objects
for row in range(0, len(policy)):
    date = str(policy.loc[row, "Date"])
    if re.search("\d+-.*", date) == None:
        policy.drop(axis = 0, index = row, inplace = True)
policy["Date"] = pd.to_datetime(policy["Date"])
        
# change column names
policy.columns = policy.columns.str.lower()
policy.columns = policy.columns.str.title()

# sort and reorganize data
policy = policy.sort_values(by = ["State", "Date"])
policy = policy.reset_index(drop = True)
policy.insert(1, "Date", policy.pop("Date"))

# OPTIONAL: subset policies
# policy = policy[policy.Policy_Category.isin([])]

In [12]:
# the final data
policy.head()

,State,Date,Policy,Policy_Category,Policy_Stage
0,Alabama,2020-03-13,STEMERG,state_of_emergency,start
1,Alabama,2020-03-13,VISITPER,incarceration,start
2,Alabama,2020-03-13,VISITATT,incarceration,start
3,Alabama,2020-03-16,TLHLMED,healthcare_delivery,start
4,Alabama,2020-03-16,WV_WTPRD,unemployment,start


<h3><center>4. GDP Data</center></h3>

### Process
<font size="3">
<ol>
    <li>Pivot the data longer so that every month has its own row</li>
    <li>Change column names and data types</li>
    <li>Sort and clean up data</li>
</ol>

In [13]:
# the head of the initial data
gdp.head()

,GeoFips,GeoName,2019:Q4-2020:Q1,2020:Q1-:Q2,2020:Q2-:Q3,2020:Q3-:Q4,2020:Q4-2021:Q1,2021:Q1-:Q2,2021:Q2-:Q3,2021:Q3-:Q4,2021:Q4-2022:Q1,2022:Q1-:Q2
0,00000,United States,-4.6,-29.9,35.3,3.9,6.3,7.0,2.7,7.0,-1.6,-0.6
1,01000,Alabama,-0.9,-29.5,42.5,0.4,5.2,5.6,0.3,5.8,-0.4,-0.9
2,02000,Alaska,-4.1,-29.8,14.2,7.3,-2.9,0.1,1.8,1.6,-15.3,-0.9
3,04000,Arizona,-2.5,-20.5,30.6,8.1,1.8,6.4,4.4,10.3,-2.2,-1.9
4,05000,Arkansas,2.8,-24.1,33.6,1.1,7.5,3.8,2.2,3.8,6.5,-3.0


In [14]:
# pivot the data longer
gdp = pd.melt(gdp, id_vars = ["GeoName", "GeoFips"], var_name = "Date", value_name = "Gdp_Change")

# change column names
gdp.rename(columns = {"GeoName": "State", "GeoFips": "Fips"}, inplace = True)

# change Date values to the correct format
date_dict = {"2019:Q4-2020:Q1": "2020-01-01", 
             "2020:Q1-:Q2": "2020-04-01", 
             "2020:Q2-:Q3": "2020-07-01", 
             "2020:Q3-:Q4": "2020-10-01",
             "2020:Q4-2021:Q1": "2021-01-01", 
             "2021:Q1-:Q2": "2021-04-01", 
             "2021:Q2-:Q3": "2021-07-01", 
             "2021:Q3-:Q4": "2021-10-01",
             "2021:Q4-2022:Q1": "2022-01-01",
             "2022:Q1-:Q2": "2022-04-01"}
gdp["Date"] = [date_dict[date] for date in gdp["Date"]]

# copy and merge the data three times so that the interval between observations is one month instead of one quarter
gdp = pd.concat([gdp, gdp, gdp])

# sort values
gdp = gdp.sort_values(by = ["State", "Date"])
gdp = gdp.reset_index(drop = True)

# remove na's
gdp = gdp.dropna(subset = ["State"])

# change the dtype of variable Date
gdp["Date"] = pd.to_datetime(gdp["Date"])

# modify the date column so that every value is one month apart from each other
from dateutil.relativedelta import relativedelta
count = 0
for row in range(0, len(gdp)):
    if count > 0:
        gdp.loc[row, "Date"] = gdp.loc[row - 1, "Date"] + relativedelta(months = 1)
    count += 1
    if count == 3:
        count = 0
        
# remove day information so that every "Date" value has only "year and month"
gdp["Date"] = gdp["Date"].dt.to_period("M")

In [15]:
# the final data
gdp.head()

,State,Fips,Date,Gdp_Change
0,Alabama,01000,2020-01,-0.9
1,Alabama,01000,2020-02,-0.9
2,Alabama,01000,2020-03,-0.9
3,Alabama,01000,2020-04,-29.5
4,Alabama,01000,2020-05,-29.5


<h3><center>5. Covid-19 Data</h3></center>

### Process
<font size="3">
<ol>
    <li>Add two variables for daily new cases and daily new deaths</li>
    <li>Change column names and dtypes as needed</li>
    <li>Merge observations from the same month so that every month only has one corresponding row</li>
</ol>

In [16]:
# the head of the initial data
covid.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [17]:
# create two columns for daily new cases and deaths
covid["New_Cases"] = np.array(0)
covid["New_Deaths"] = np.array(0)

for state in covid.state.unique():
    cumulative_cases = np.array(covid.loc[covid.state == state, "cases"].diff())
    new_cases = cumulative_cases
    new_cases[0] = cumulative_cases[0]
    
    cumulative_deaths = np.array(covid.loc[covid.state == state, "deaths"].diff())
    new_deaths = cumulative_deaths
    new_deaths[0] = cumulative_deaths[0]
    
    covid.loc[covid.state == state, "New_Cases"] = new_cases
    covid.loc[covid.state == state, "New_Deaths"] = new_deaths

# capitalize column names
covid.columns = covid.columns.str.title()

# change the format and dtype of variable Date
covid["Date"] = pd.to_datetime(covid["Date"])
covid["Date"] = covid["Date"].dt.to_period("M")

# combine rows that are in the same month so that each month has only 1 corresponding row
temp_covid = covid.pivot_table(index = ["State", "Date"], values = ["New_Cases", "New_Deaths"], aggfunc = "sum")
temp_covid["Cum_Cases"] = temp_covid.groupby(["State", "Date"]).New_Cases.cumsum()
temp_covid["Cum_Deaths"] = temp_covid.groupby(["State", "Date"]).New_Deaths.cumsum()
covid = temp_covid.reset_index()

In [18]:
# the final data
covid.head()

,State,Date,New_Cases,New_Deaths,Cum_Cases,Cum_Deaths
0,Alabama,2020-03,993.0,14.0,993.0,14.0
1,Alabama,2020-04,6069.0,258.0,6069.0,258.0
2,Alabama,2020-05,10884.0,358.0,10884.0,358.0
3,Alabama,2020-06,20093.0,320.0,20093.0,320.0
4,Alabama,2020-07,49678.0,630.0,49678.0,630.0


<h3><center>6. Unemployment Data</h3></center>

### Process
<font size="3">
<ol>
    <li>Using codebook, turn series id to the corresponding state</li>
    <li>Turn date values into a more computer-friendly format</li>
</ol>

In [19]:
# the head of the initial data
unemp.head()

,Series ID,Year,Period,Label,Value
0,LASST010000000000003,2020,M01,2020 Jan,3.3
1,LASST010000000000003,2020,M02,2020 Feb,3.4
2,LASST010000000000003,2020,M03,2020 Mar,3.6
3,LASST010000000000003,2020,M04,2020 Apr,13.7
4,LASST010000000000003,2020,M05,2020 May,10.4


In [20]:
# import area codes to be used
with open("raw_data/code.txt") as file:
    lines = file.readlines()

df = list()
for line in range(1, len(lines)):
    row = list(lines[line].split("\t"))
    df.append(row)
df = pd.DataFrame(df)
    
code = pd.DataFrame({"Area_Code": df.loc[:, 1],
                           "Area": df.loc[:, 2]})
code.head()

,Area_Code,Area
0,ST0100000000000,Alabama
1,ST0200000000000,Alaska
2,ST0400000000000,Arizona
3,ST0500000000000,Arkansas
4,ST0600000000000,California


In [21]:
# merge unemployment data with its codebook
unemp["Series ID"] = unemp["Series ID"].str.slice(3, 18)
unemp = unemp.merge(code, how = "left", left_on = "Series ID", right_on = "Area_Code")

# drop irrelevant columns
unemp = unemp.drop(axis = 1, labels = ["Series ID", "Area_Code", "Year", "Period"])

# rename and reordercolumns
unemp.rename(columns = {"Label": "Date",
                        "Value": "Pct_Unemployed",
                        "Area": "State"}, inplace = True)

unemp.insert(0, "State", unemp.pop("State"))
unemp.insert(1, "Date", unemp.pop("Date"))

# change values in Date column into a more computer-friendly format
from datetime import datetime
unemp["Date"] = [datetime.strptime(date, "%Y %b") for date in unemp["Date"]]
unemp["Date"] = pd.to_datetime(unemp["Date"]).dt.to_period("M")


In [22]:
# the final data
unemp.head()

,State,Date,Pct_Unemployed
0,Alabama,2020-01,3.3
1,Alabama,2020-02,3.4
2,Alabama,2020-03,3.6
3,Alabama,2020-04,13.7
4,Alabama,2020-05,10.4


<h3><center>7. Personal Income Data</h3></center>

### Process
<font size="3">
<ol>
    <li>Pivot the data longer so that date becomes a column</li>
    <li>Change column names & data types & data formats</li>
    <li>Pivot the data wider so that income is separated based on types</li>
</ol>

In [23]:
# the head of the initial data
income.head()

,GeoFips,GeoName,LineCode,Description,2019:Q4-2020:Q1,2020:Q1-:Q2,2020:Q2-:Q3,2020:Q3-:Q4,2020:Q4-2021:Q1,2021:Q1-:Q2,2021:Q2-:Q3,2021:Q3-:Q4,2021:Q4-2022:Q1,2022:Q1-:Q2
0,00000,United States,NaN,Income by place of residence,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00000,United States,10.0,"Personal income (millions of dollars, seasonal...",1,7.6,-2.3,-1.1,11.7,-5.4,0.4,0.8,0.8,1.4
2,00000,United States,11.0,Nonfarm personal income 1/,1,7.7,-2.3,-1.3,11.9,-5.6,0.5,0.9,0.6,1.3
3,00000,United States,12.0,Farm income 2/,-4.4,-17,30.1,41,-35,62.9,-6.2,-18,35.9,18.6
4,00000,United States,20.0,"Population (midperiod, persons) 3/",0.1,0,0,0,0,0,0.1,0.1,0,0.1


In [24]:
# drop irrelevant columns
income = income.drop(axis = 1, labels = ["GeoFips", "LineCode"])

# pivot the data longer 
income = pd.melt(income, id_vars = ["GeoName", "Description"], var_name = "Date", value_name = "Income_Change")

# change date values to the correct format
date_dict = {"2019:Q4-2020:Q1": "2020-01-01", 
             "2020:Q1-:Q2": "2020-04-01", 
             "2020:Q2-:Q3": "2020-07-01", 
             "2020:Q3-:Q4": "2020-10-01",
             "2020:Q4-2021:Q1": "2021-01-01", 
             "2021:Q1-:Q2": "2021-04-01", 
             "2021:Q2-:Q3": "2021-07-01", 
             "2021:Q3-:Q4": "2021-10-01",
             "2021:Q4-2022:Q1": "2022-01-01",
             "2022:Q1-:Q2": "2022-04-01"}

income["Date"] = [date_dict[date] for date in income["Date"]]

# drop na values
income = income.dropna(subset = ["Income_Change"])

# modify column names
income = income.rename(columns = {"GeoName": "State", "Description": "Income_Type"})

# "copy-paste" the data three times so that each month gets its own row
income = pd.concat([income, income, income])

# sort data by State then Date
income = income.sort_values(by = ["State", "Date"])
income = income.reset_index(drop = True)

# change the dtype of date column
income["Date"] = pd.to_datetime(income["Date"])

# modify date intervals so that each date entry is exactly one month apart from each other
from dateutil.relativedelta import relativedelta
count = 0
for row in range(0, len(income)):
    if count > 0:
        income.loc[row, "Date"] = income.loc[row - 1, "Date"] + relativedelta(months = 1)
    count += 1
    if count == 3:
        count = 0

        
# pivot the data wider so that each type of income has its own column
income = income.pivot(index = ["State", "Date"], columns = "Income_Type", values = "Income_Change")

# reset index
income = income.reset_index()

# rename columns 
income = pd.DataFrame({"State": income["State"], 
                       "Date": income["Date"], 
                       "Income_Change": income.iloc[:, 4],
                       "Adjusted_Income_Change": income["Personal income (millions of dollars, seasonally adjusted)"],
                       "Farm_Income_Change": income.iloc[:, 3],
                       "Nonfarm_Income_Change": income.iloc[:, 2]})


# change Date column to the correct format
income["Date"] = income["Date"].dt.to_period("M")


In [25]:
# the final data
income.head()

,State,Date,Income_Change,Adjusted_Income_Change,Farm_Income_Change,Nonfarm_Income_Change
0,Alabama,2020-01,1.6,1.8,1.7,13
1,Alabama,2020-02,1.6,1.8,1.7,13
2,Alabama,2020-03,1.6,1.8,1.7,13
3,Alabama,2020-04,8.6,8.7,9.3,(NM)
4,Alabama,2020-05,8.6,8.7,9.3,(NM)


<h3><center>8. Population Data</h3></center>

### Process
<font size="3">
<ol>
    <li>Pivot the dataset longer so that each month has its own row</li>
    <li>Sort and reorder data</li>
    <li>Adjust the dtype and format of the Date column</li>
</ol>

In [26]:
# the head of the initial data
pop.head()

,GeoFips,GeoName,2020:Q1,2020:Q2,2020:Q3,2020:Q4,2021:Q1,2021:Q2,2021:Q3,2021:Q4,2022:Q1,2022:Q2
0,00000,United States,331295939.0,331448217.0,331596557.0,331734262.0,331706294.0,331776226.0,332049982.0,332336782.0,332502197.0,332693300.0
1,01000,Alabama,5021627.0,5024115.0,5027375.0,5031760.0,5033508.0,5036858.0,5043548.0,5050555.0,5055254.0,5060373.0
2,02000,Alaska *,733708.0,732864.0,732535.0,732666.0,732497.0,732509.0,732860.0,733229.0,733455.0,733722.0
3,04000,Arizona,7139186.0,7164176.0,7191495.0,7217049.0,7238812.0,7262623.0,7291158.0,7320058.0,7346243.0,7372931.0
4,05000,Arkansas,3010767.0,3011622.0,3014347.0,3018135.0,3020316.0,3023500.0,3028649.0,3033994.0,3037957.0,3042210.0


In [27]:
# make sure State names are correctly formatted
pop["GeoName"] = [str(name).replace(" *", "") for name in pop["GeoName"]]

# drop irrelevant columns
pop = pop.drop(axis = 1, labels = "GeoFips")

# rename columns
pop.rename(columns = {
    "GeoName": "State",
}, inplace = True)

# pivot the data longer so that every month has its own row
pop = pd.melt(pop, id_vars = "State", var_name = "Date", value_name = "Population")

# change Date values to the correct format
date_dict = {"2020:Q1": "2020-01-01", 
             "2020:Q2": "2020-04-01", 
             "2020:Q3": "2020-07-01", 
             "2020:Q4": "2020-10-01",
             "2021:Q1": "2021-01-01", 
             "2021:Q2": "2021-04-01", 
             "2021:Q3": "2021-07-01", 
             "2021:Q4": "2021-10-01",
             "2022:Q1": "2022-01-01",
             "2022:Q2": "2022-04-01"}
pop["Date"] = [date_dict[date] for date in pop["Date"]]

# "copy-paste" the dataset three times so that each month has its own row
pop = pd.concat([pop, pop, pop])

# sort and reorder data
pop = pop.sort_values(by = ["State", "Date"])
pop = pop.reset_index(drop = True)
pop.insert(1, "Date", pop.pop("Date")) # the data happens to call a method also named "pop" lol

# change the dtype and format of the Date column
from dateutil.relativedelta import relativedelta
pop["Date"] = pd.to_datetime(pop["Date"])
count = 0
for row in range(0, len(pop)):
    if count > 0:
        pop.loc[row, "Date"] = pop.loc[row - 1, "Date"] + relativedelta(months = 1)
    count += 1
    if count == 3:
        count = 0

pop["Date"] = pop["Date"].dt.to_period("M")

In [28]:
# the final data
pop.head()

,State,Date,Population
0,Alabama,2020-01,5021627.0
1,Alabama,2020-02,5021627.0
2,Alabama,2020-03,5021627.0
3,Alabama,2020-04,5024115.0
4,Alabama,2020-05,5024115.0


<h1><center>Merge Data</center></h1>

In [29]:
# merge all data
temp = pd.merge_ordered(health_state, covid, on = ["State", "Date"], how = "left")
temp = pd.merge_ordered(temp, gdp, on = ["State", "Date"], how = "left")
temp = pd.merge_ordered(temp, income, on = ["State", "Date"], how = "left")
temp = pd.merge_ordered(temp, unemp, on = ["State", "Date"], how = "left")
temp = temp.merge(postcode, on = "State", how = "left")
final = pd.merge_ordered(temp, pop, on = ["State", "Date"], how = "left")

In [30]:
# add two new variables Postcode Pct_Infected and Pct_Mortality
Pct_Infection = final["New_Cases"] / final["Population"] * 100
Pct_Mortality = final["New_Deaths"] / final["Population"] * 100
final.insert(9, "Pct_Infection", Pct_Infection)
final.insert(10, "Pct_Mortality", Pct_Mortality)

# reorder columns
final.insert(1, "Postcode", final.pop("Postcode"))
final.insert(2, "Fips", final.pop("Fips"))

In [31]:
# final merged data
final.head()

,State,Postcode,Fips,Date,Pct_Anxiety,Pct_Depression,Pct_Anxiety_Or_Depression,New_Cases,New_Deaths,Cum_Cases,Cum_Deaths,Pct_Infection,Pct_Mortality,Gdp_Change,Income_Change,Adjusted_Income_Change,Farm_Income_Change,Nonfarm_Income_Change,Pct_Unemployed,Population
0,Alabama,AL,01000,2020-04,25.600,18.600000,30.300,6069.0,258.0,6069.0,258.0,0.120797,0.005135,-29.5,8.6,8.7,9.3,(NM),13.7,5024115.0
1,Alabama,AL,01000,2020-05,26.175,22.850000,30.525,10884.0,358.0,10884.0,358.0,0.216635,0.007126,-29.5,8.6,8.7,9.3,(NM),10.4,5024115.0
2,Alabama,AL,01000,2020-06,32.500,27.100000,37.225,20093.0,320.0,20093.0,320.0,0.399931,0.006369,-29.5,8.6,8.7,9.3,(NM),8.7,5024115.0
3,Alabama,AL,01000,2020-07,36.400,30.566667,41.000,49678.0,630.0,49678.0,630.0,0.988150,0.012531,42.5,-4.1,-4,-4.2,(NM),7.7,5027375.0
4,Alabama,AL,01000,2020-08,30.600,27.700000,37.200,38335.0,602.0,38335.0,602.0,0.762525,0.011974,42.5,-4.1,-4,-4.2,(NM),6.6,5027375.0


In [32]:
# final health data
health.head()

,State,Date,Group,Subgroup,Pct_Anxiety,Pct_Depression,Pct_Anxiety_Or_Depression
0,United States,2020-04-29,By Age,18 - 29 years,40.2,32.7,46.8
1,United States,2020-05-09,By Age,18 - 29 years,42.0,36.2,47.4
2,United States,2020-05-16,By Age,18 - 29 years,38.6,36.7,47.7
3,United States,2020-05-23,By Age,18 - 29 years,39.1,36.7,46.6
4,United States,2020-05-30,By Age,18 - 29 years,42.2,36.8,49.3


In [33]:
# final policy data
policy.head()

,State,Date,Policy,Policy_Category,Policy_Stage
0,Alabama,2020-03-13,STEMERG,state_of_emergency,start
1,Alabama,2020-03-13,VISITPER,incarceration,start
2,Alabama,2020-03-13,VISITATT,incarceration,start
3,Alabama,2020-03-16,TLHLMED,healthcare_delivery,start
4,Alabama,2020-03-16,WV_WTPRD,unemployment,start


<h1><center>Export Data</center></h1>

In [35]:
final.to_excel("final_data/fulldata_final.xlsx")
health.to_excel("final_data/health_final.xlsx")
policy.to_excel("final_data/policy_final.xlsx")